# Como remodelar o layout das tabelas

Inicie importando o panda e lendo os arquivos titanic.csv e air_quality_no2.csv

In [ ]:
import pandas as pd

titanic = pd.read_csv("data/titanic.csv")
titanic.head()


In [ ]:
air_quality = pd.read_csv("data/air_quality_long.csv", index_col="date.utc", parse_dates=True)
air_quality.head()

# Classificar linhas da tabela

Quero classificar os dados do Titanic de acordo com a idade dos passageiros.

In [ ]:
titanic.sort_values(by="Age").head()

Quero classificar os dados do Titanic de acordo com a classe da cabine e a idade em ordem decrescente.

In [ ]:
titanic.sort_values(by=['Pclass', 'Age'], ascending=False).head()

Com DataFrame.sort_values(), as linhas da tabela são classificadas de acordo com a(s) coluna(s) definida(s).  
O índice seguirá a ordem das linhas.

# Formato de tabela longa para larga

Vamos usar um pequeno subconjunto do conjunto de dados de qualidade do ar.  
Concentramo-nos em usar apenas os dados das duas primeiras medições de cada local (ou seja, a cabeça de cada grupo).  
O subconjunto de dados será chamado de no2_subset.

In [ ]:
no2 = air_quality[air_quality["parameter"] == "no2"]
no2_subset = no2.sort_index().groupby(["location"]).head(2)
no2_subset

Quero os valores das três estações em colunas separadas, uma ao lado da outra.  

A função pivot(), é puramente uma reformulação dos dados:  
* é necessário um único valor para cada combinação de índice/coluna.

In [ ]:
no2_subset.pivot(columns="location", values="value")

Como o pandas oferece suporte imediato à plotagem de múltiplas colunas (veja o tutorial de plotagem ),  
a conversão do formato de tabela longa para larga permite a plotagem de diferentes séries temporais ao mesmo tempo:

In [ ]:
no2.head()

In [ ]:
no2.pivot(columns="location", values="value").plot()

Quando o indexparâmetro não é definido, o índice existente (rótulos de linha) é usado.

# Tabela dinâmica

Quero as concentrações médias para e em cada uma das estações em forma de tabela.

No caso de pivot(), os dados são apenas reorganizados.  
Quando vários valores precisam ser agregados (neste caso específico, os valores em diferentes intervalos de tempo),  
pivot_table() pode ser usado, fornecendo uma função de agregação (por exemplo, média) sobre como combinar esses valores.

In [ ]:
air_quality.pivot_table(
    values="value", index="location", columns="parameter", aggfunc="mean"
)

Tabela dinâmica é um conceito bem conhecido em softwares de planilhas.  
Se estiver interessado nas margens de linha/coluna (subtotais) de cada variável, defina o marginsparâmetro como True:

In [ ]:
air_quality.pivot_table(
    values="value",
    index="location",
    columns="parameter",
    aggfunc="mean",
    margins=True,
)

Caso você esteja se perguntando, pivot_table()está de fato diretamente ligado a groupby().  
O mesmo resultado pode ser obtido agrupando em parametere location:

        air_quality.groupby(["parameter", "location"])[["value"]].mean()

# Formato largo para longo

Começando novamente a partir da tabela de formato amplo criada na seção anterior,  
adicionamos um novo índice ao DataFramewith reset_index().

In [ ]:
no2_pivoted = no2.pivot(columns="location", values="value").reset_index()
no2_pivoted.head()

Quero coletar toda a qualidade do ar NO² medições em uma única coluna (formato longo).

O método pandas.melt(), em DataFrame, converte a tabela de dados do formato amplo para o formato longo.  
Os cabeçalhos das colunas se tornam os nomes das variáveis ​​em uma coluna recém-criada.

In [ ]:
no_2 = no2_pivoted.melt(id_vars="date.utc")
no_2.head()

A solução é a versão resumida de como aplicar pandas.melt().  
O método fundirá todas as colunas NÃO mencionadas id_varsem duas colunas:  

* Uma coluna com os nomes dos cabeçalhos 
* Uma coluna com os valores propriamente ditos  

Esta última coluna recebe, por padrão, o nome value.

Os parâmetros passados para o pandas.melt(), ​​podem ser definidos com mais detalhes:

In [ ]:
no_2 = no2_pivoted.melt(
    id_vars="date.utc",
    value_vars=["BETR801", "FR04014", "London Westminster"],
    value_name="NO_2",
    var_name="id_location",
)
no_2.head()

Os parâmetros adicionais têm os seguintes efeitos:

* value_vars define quais colunas devem ser fundidas
* value_name fornece um nome de coluna personalizado para a coluna de valores em vez do nome de coluna padrão value
* var_name fornece um nome de coluna personalizado para a coluna que coleta os nomes dos cabeçalhos das colunas. 
    Caso contrário, ele usa o nome do índice ou um nome padrãovariable

Portanto, os argumentos value_namee var_name são apenas nomes definidos pelo usuário para as duas colunas geradas.  
As colunas a serem fundidas são definidas por id_varse value_vars.

**LEMBRAR**

* A classificação por uma ou mais colunas é suportada por sort_values.
* A pivotfunção é puramente de reestruturação dos dados e pivot_tablesuporta agregações.
* O inverso de pivot(formato longo para formato largo) é melt(formato largo para formato longo).